In [145]:
!pip install geopandas
!pip install shapely

In [146]:
import geopandas as gpd
import shapely
import pandas as pd
import numpy as np

In [147]:
geojson_df = gpd.read_file("/ConnecticutAddress-Geocoded.geojson")


In [148]:
geojson_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 69796 entries, 0 to 69795
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Address       69796 non-null  object  
 1   Town          69796 non-null  object  
 2   State         69796 non-null  object  
 3   field_4       69796 non-null  object  
 4   field_5       69796 non-null  object  
 5   field_6       69796 non-null  object  
 6   field_7       69796 non-null  object  
 7   field_8       69796 non-null  object  
 8   field_9       69796 non-null  object  
 9   field_10      69796 non-null  object  
 10  field_11      69796 non-null  object  
 11  field_12      69796 non-null  object  
 12  field_13      69796 non-null  object  
 13  field_14      69796 non-null  object  
 14  field_15      69796 non-null  object  
 15  result_num    69796 non-null  object  
 16  osm_id        69796 non-null  object  
 17  display_name  69796 non-null  object  
 18

In [149]:
geojson_df[['Latitude', 'Longitude']] = geojson_df['latlong'].str.split(',', expand=True)
# Convert 'Latitude' and 'Longitude' columns to numeric type
geojson_df['Latitude'] = pd.to_numeric(geojson_df['Latitude'])
geojson_df['Longitude'] = pd.to_numeric(geojson_df['Longitude'])
# Drop the original 'latlong' column
geojson_df.drop(columns=['latlong'], inplace=True)

In [150]:
towns = geojson_df['Town'].copy()
elements_to_remove = ['136', '21']
mask = np.logical_not(np.isin(towns, elements_to_remove))
filtered_towns = towns[mask]
print(len(filtered_towns.unique()))

169


In [151]:
real_estate_df = pd.read_csv("/Real_Estate_Sales_2001-2021_GL_20240304.csv")
real_estate_df.count()

<ipython-input-151-da2ff1d2f238>:1: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  real_estate_df = pd.read_csv("/Real_Estate_Sales_2001-2021_GL_20240304.csv")


Serial Number       1054159
List Year           1054159
Date Recorded       1054157
Town                1054159
Address             1054108
Assessed Value      1054159
Sale Amount         1054159
Sales Ratio         1054159
Property Type        671713
Residential Type     660275
Non Use Code         302242
Assessor Remarks     161472
OPM remarks           11564
Location             254643
dtype: int64

In [152]:
properties = ["Single Family", "Two Family", "Three Family", "Four Family", "Condo"]
real_estate_df = real_estate_df[real_estate_df['Residential Type'].isin(properties)]
real_estate_df.count()

Serial Number       660275
List Year           660275
Date Recorded       660275
Town                660275
Address             660271
Assessed Value      660275
Sale Amount         660275
Sales Ratio         660275
Property Type       660275
Residential Type    660275
Non Use Code        192329
Assessor Remarks    134811
OPM remarks          10470
Location            175126
dtype: int64

In [153]:
real_estate_df['Location'].count()

175126

In [154]:
def create_point(latitude, longitude):
    return f"POINT ({longitude} {latitude})"

In [155]:
null_location_df = real_estate_df[real_estate_df['Location'].isnull()]
merged_df = pd.merge(null_location_df, geojson_df, on='Address', how='left')
merged_df['Location'] = merged_df.apply(lambda row: create_point(row['Latitude'], row['Longitude']), axis=1)
merged_df.drop_duplicates(subset=['Address', 'Location'], keep='first', inplace=True)

In [156]:
merged_df = merged_df[merged_df['Location'] != 'POINT (nan nan)']
merged_df['Location'].count()
merged_df.rename(columns={'Town_x': 'Town'}, inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43763 entries, 0 to 492250
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Serial Number     43763 non-null  int64   
 1   List Year         43763 non-null  int64   
 2   Date Recorded     43763 non-null  object  
 3   Town              43763 non-null  object  
 4   Address           43763 non-null  object  
 5   Assessed Value    43763 non-null  float64 
 6   Sale Amount       43763 non-null  float64 
 7   Sales Ratio       43763 non-null  float64 
 8   Property Type     43763 non-null  object  
 9   Residential Type  43763 non-null  object  
 10  Non Use Code      8403 non-null   object  
 11  Assessor Remarks  5365 non-null   object  
 12  OPM remarks       746 non-null    object  
 13  Location          43763 non-null  object  
 14  Town_y            43763 non-null  object  
 15  State             43763 non-null  object  
 16  field_4           43763 no

<ipython-input-156-0da28d5bdb2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.rename(columns={'Town_x': 'Town'}, inplace=True)


In [157]:
# Columns to drop
columns_to_drop = ['Town_y', 'State', 'field_4', 'field_5', 'field_6', 'field_7',
                   'field_8', 'field_9', 'field_10', 'field_11', 'field_12', 'field_13',
                   'field_14', 'field_15', 'result_num', 'osm_id', 'display_name',
                   'category', 'type', 'geometry', 'Latitude', 'Longitude']

# Drop the specified columns
merged_df.drop(columns=columns_to_drop, inplace=True)


In [158]:
merged_df.count()

Serial Number       43763
List Year           43763
Date Recorded       43763
Town                43763
Address             43763
Assessed Value      43763
Sale Amount         43763
Sales Ratio         43763
Property Type       43763
Residential Type    43763
Non Use Code         8403
Assessor Remarks     5365
OPM remarks           746
Location            43763
dtype: int64

In [159]:
merged_real_estate_df = pd.merge(real_estate_df, merged_df, on='Address', how='left', suffixes=('_existing', '_merged'))
null_location_mask = merged_real_estate_df['Location_existing'].isnull()
merged_real_estate_df.loc[null_location_mask, 'Location_existing'] = merged_real_estate_df.loc[null_location_mask, 'Location_merged']
merged_real_estate_df.drop(columns=['Location_merged'], inplace=True)

In [160]:
merged_real_estate_df = merged_real_estate_df[merged_real_estate_df['Location_existing'].notnull()]
merged_real_estate_df.count()

Serial Number_existing       243680
List Year_existing           243680
Date Recorded_existing       243680
Town_existing                243680
Address                      243680
Assessed Value_existing      243680
Sale Amount_existing         243680
Sales Ratio_existing         243680
Property Type_existing       243680
Residential Type_existing    243680
Non Use Code_existing         62874
Assessor Remarks_existing     45174
OPM remarks_existing           4638
Location_existing            243680
Serial Number_merged          75061
List Year_merged              75061
Date Recorded_merged          75061
Town_merged                   75061
Assessed Value_merged         75061
Sale Amount_merged            75061
Sales Ratio_merged            75061
Property Type_merged          75061
Residential Type_merged       75061
Non Use Code_merged           15186
Assessor Remarks_merged        9636
OPM remarks_merged             1493
dtype: int64

In [161]:
merged_real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243680 entries, 0 to 663538
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Serial Number_existing     243680 non-null  int64  
 1   List Year_existing         243680 non-null  int64  
 2   Date Recorded_existing     243680 non-null  object 
 3   Town_existing              243680 non-null  object 
 4   Address                    243680 non-null  object 
 5   Assessed Value_existing    243680 non-null  float64
 6   Sale Amount_existing       243680 non-null  float64
 7   Sales Ratio_existing       243680 non-null  float64
 8   Property Type_existing     243680 non-null  object 
 9   Residential Type_existing  243680 non-null  object 
 10  Non Use Code_existing      62874 non-null   object 
 11  Assessor Remarks_existing  45174 non-null   object 
 12  OPM remarks_existing       4638 non-null    object 
 13  Location_existing          243680 

In [162]:
# Columns to drop
columns_to_drop = ['Serial Number_merged', 'List Year_merged', 'Date Recorded_merged',
                   'Town_merged', 'Assessed Value_merged', 'Sale Amount_merged',
                   'Sales Ratio_merged', 'Property Type_merged', 'Residential Type_merged',
                   'Non Use Code_merged', 'Assessor Remarks_merged', 'OPM remarks_merged']

# Drop the specified columns
merged_real_estate_df.drop(columns=columns_to_drop, inplace=True)

In [163]:
# Remove the '_existing' suffix from column names
merged_real_estate_df.rename(columns=lambda x: x.replace('_existing', ''), inplace=True)

In [144]:
from google.colab import files
merged_real_estate_df.to_csv('AIT664-Group001-Connecticut-RealEstateData-Preprocessed-Part01.csv', index=False)
files.download('AIT664-Group001-Connecticut-RealEstateData-Preprocessed-Part01.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>